In [ ]:
import pickle
import torch.nn as nn
import torch
import pandas as pd
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.metrics import mean_absolute_error,r2_score, mean_squared_error
import random
import os

def set_deterministic(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_deterministic(42)

## 0. Dataset and DataLoader

In [ ]:
name='CAISO_zone_1_.csv'
surrogate_model_path = 'transformer_wind_power_model2_CAISO_zone_1_.pth'
with open('weather_scaler.pkl', 'rb') as f:
    weather_scaler = pickle.load(f)

class WindPowerDataset(Dataset):
    def __init__(self, csv_file, wind_power_scaler=None, weather_scaler=None, save_scalers=False):
        self.data = pd.read_csv(csv_file)

        self.data = self.data.iloc[::5, :].reset_index(drop=True)

        self.original_wind_power_std = self.data.iloc[:, 2].std()
        self.original_weather_std = self.data.iloc[:, 4:12].std()

        if wind_power_scaler is None or weather_scaler is None:
            self.wind_power_scaler = MinMaxScaler()
            self.weather_scaler = MinMaxScaler()

            self.data.iloc[:, 2] = self.wind_power_scaler.fit_transform(self.data.iloc[:, 2].values.reshape(-1, 1)).squeeze()

            self.data.iloc[:, 4:12] = self.weather_scaler.fit_transform(self.data.iloc[:, 4:12])
            
            if save_scalers:
                with open('wind_power_scaler.pkl', 'wb') as f:
                    pickle.dump(self.wind_power_scaler, f)
                with open('weather_scaler.pkl', 'wb') as f:
                    pickle.dump(self.weather_scaler, f)
        else:
            self.wind_power_scaler = wind_power_scaler
            self.weather_scaler = weather_scaler
            self.data.iloc[:, 2] = self.wind_power_scaler.transform(self.data.iloc[:, 2].values.reshape(-1, 1)).squeeze()
            self.data.iloc[:, 4:12] = self.weather_scaler.transform(self.data.iloc[:, 4:12])
    
    def __len__(self):
        return len(self.data) - 312  # 288 (1440/5) + 24 (120/5)
    
    def __getitem__(self, idx):
        wind_power_history = self.data.iloc[idx:idx + 288, 2].values.astype(float)
        future_weather = self.data.iloc[idx + 288:idx + 312, 4:12].values.astype(float)
        future_wind_power = self.data.iloc[idx + 312, 2]
        return torch.tensor(wind_power_history, dtype=torch.float32), \
               torch.tensor(future_weather, dtype=torch.float32), \
               torch.tensor(future_wind_power, dtype=torch.float32)
    
    def get_original_stds(self):
        return {
            'original_wind_power_std': self.original_wind_power_std,
            'original_weather_std': self.original_weather_std.to_dict()
        }

dataset = WindPowerDataset(name, save_scalers=True)
weather_stds = dataset.get_original_stds()
weather_stds_array = np.array(list(weather_stds['original_weather_std'].values()))  
weather_data = pd.read_csv(name)
weather_data = weather_data.iloc[::5, :].reset_index(drop=True)
weather_mean = weather_data.iloc[:, 4:12].mean()
wind_power_scaler=dataset.wind_power_scaler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=32
test_split=0.2
test_size = int(len(dataset) * test_split)
train_size = len(dataset) - test_size
train_dataset = Subset(dataset, list(range(train_size)))
test_dataset = Subset(dataset, list(range(train_size, len(dataset))))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## 1. Load surrogate model and target model

### traget model

In [ ]:
class WindPowerPredictor(nn.Module):
    def __init__(self):
        super(WindPowerPredictor, self).__init__()
        self.lstm_wind = nn.LSTM(input_size=1, hidden_size=128, num_layers=1, batch_first=True)
        self.lstm_weather = nn.LSTM(input_size=8, hidden_size=128, num_layers=1, batch_first=True)
        self.fc = nn.Linear(256, 1)
        self.relu=nn.ReLU()
    
    def forward(self, wind_history, weather_future):
        wind_history = wind_history.unsqueeze(-1)
        _, (hn_wind, _) = self.lstm_wind(wind_history)
        _, (hn_weather, _) = self.lstm_weather(weather_future)
        hn_wind = hn_wind[-1, :, :]
        hn_weather = hn_weather[-1, :, :]
        combined = torch.cat((hn_wind, hn_weather), dim=1)
        output = self.fc(combined)
        output = self.relu(output)
        return output
    
def load_model(model_path, device='cpu'):
    model = WindPowerPredictor().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tar_model_path1 = 'wind_caiso_lstm_sigmoid_version2.pth'
tar_model1 = load_model(tar_model_path1, device)

class WindPowerPredictor(nn.Module):  
    def __init__(self):  
        super(WindPowerPredictor, self).__init__()  
        self.gru_wind = nn.GRU(input_size=1, hidden_size=128, num_layers=1, batch_first=True)  
        self.gru_weather = nn.GRU(input_size=8, hidden_size=128, num_layers=1, batch_first=True)  
        self.fc = nn.Linear(256, 1)  
        self.sigmoid = nn.Sigmoid()
      
    def forward(self, wind_history, weather_future):  
        wind_history = wind_history.unsqueeze(-1)  
        _, hn_wind = self.gru_wind(wind_history)  
        _, hn_weather = self.gru_weather(weather_future)  
        hn_wind = hn_wind[-1, :, :]  
        hn_weather = hn_weather[-1, :, :]  
        combined = torch.cat((hn_wind, hn_weather), dim=1)  
        output = self.fc(combined)  
        output = self.sigmoid(output)
        return output  

tar_model_path2 = 'wind_gru_caiso_sigmoid_version2.pth'
tar_model2 = load_model(tar_model_path2, device)

class WindPowerPredictorV2(nn.Module):
    def __init__(self, d_model=64, nhead=4, num_layers=3, dim_feedforward=128, dropout=0.1, max_seq_len=300):
        super(WindPowerPredictorV2, self).__init__()
        self.d_model = d_model

        self.embedding_wind = nn.Linear(1, d_model)
        self.embedding_weather = nn.Linear(8, d_model)

        self.pos_encoder_wind = nn.Parameter(torch.zeros(max_seq_len, 1, d_model))
        self.pos_encoder_weather = nn.Parameter(torch.zeros(max_seq_len, 1, d_model))


        encoder_layers = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=False
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)

        self.fc_combine = nn.Sequential(
            nn.Linear(d_model * 2, d_model),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, 1)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, wind_history, weather_future):
        # wind_history: (batch_size, T1)        e.g., (32, 288)
        # weather_future: (batch_size, T2, 8)    e.g., (32, 24, 8)

        batch_size, seq_len_wind = wind_history.shape
        _, seq_len_weather, _ = weather_future.shape

        if seq_len_wind > self.pos_encoder_wind.size(0):
            raise ValueError(f"Wind sequence length {seq_len_wind} exceeds max supported length {self.pos_encoder_wind.size(0)}")
        if seq_len_weather > self.pos_encoder_weather.size(0):
            raise ValueError(f"Weather sequence length {seq_len_weather} exceeds max supported length {self.pos_encoder_weather.size(0)}")

        wind_emb = self.embedding_wind(wind_history.unsqueeze(-1))           # (B, T1, D)
        weather_emb = self.embedding_weather(weather_future)                 # (B, T2, D)

        wind_emb = wind_emb.permute(1, 0, 2)  # (T1, B, D)
        weather_emb = weather_emb.permute(1, 0, 2)  # (T2, B, D)

        wind_emb = wind_emb + self.pos_encoder_wind[:seq_len_wind]      # (T1, B, D)
        weather_emb = weather_emb + self.pos_encoder_weather[:seq_len_weather]  # (T2, B, D)

        wind_features = self.transformer_encoder(wind_emb)              # (T1, B, D)
        weather_features = self.transformer_encoder(weather_emb)        # (T2, B, D)

        wind_last = wind_features[-1, :, :]        
        weather_last = weather_features[-1, :, :]  

        combined = torch.cat((wind_last, weather_last), dim=1)  # (B, 2D)

        output = self.fc_combine(combined)  # (B, 1)
        output = self.sigmoid(output)
        return output
    
tar_model_path3 = 'wind_caiso_transformer_sigmoid_version2.pth'
tar_model3 = WindPowerPredictorV2().to(device)
state_dict = torch.load(tar_model_path3, map_location=device)
tar_model3.load_state_dict(state_dict)
tar_model3.to(device)

class TemporalConvNet_V2(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=3, dropout=0.2):
        super(TemporalConvNet_V2, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]

            padding = (kernel_size - 1) * dilation_size

            conv = nn.Conv1d(
                in_channels, out_channels, kernel_size,
                stride=1, padding=padding, dilation=dilation_size  
            )
            relu = nn.ReLU()
            drop = nn.Dropout(dropout)

            if in_channels != out_channels:
                res_conv = nn.Conv1d(in_channels, out_channels, 1)
            else:
                res_conv = None

            layers.append(nn.ModuleDict({
                'conv': conv,
                'relu': relu,
                'dropout': drop,
                'res_conv': res_conv
            }))
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        # x: (batch_size, in_channels, seq_len)
        for layer in self.layers:
            residual = x  

            x = layer['conv'](x)  
            x = layer['relu'](x)
            x = layer['dropout'](x)
            x = x[:, :, :residual.size(2)]  

            # 残差连接
            if layer['res_conv'] is not None:
                residual = layer['res_conv'](residual)
            x = x + residual
            x = layer['relu'](x)  
        return x


class WindPowerPredictorTCN_V2(nn.Module):
    def __init__(self, d_model=64, dropout=0.2, fusion_mode='concat'):
        super(WindPowerPredictorTCN_V2, self).__init__()
        self.fusion_mode = fusion_mode  # 'concat' or 'add'
        self.tcn_wind = TemporalConvNet_V2(
            num_inputs=1,
            num_channels=[d_model, d_model*2, d_model, d_model],  
            kernel_size=3,
            dropout=dropout
        )

        self.tcn_weather = TemporalConvNet_V2(
            num_inputs=8,
            num_channels=[d_model, d_model*2, d_model, d_model],
            kernel_size=3,
            dropout=dropout
        )

        if fusion_mode == 'concat':
            fc_input_dim = d_model * 2
        elif fusion_mode == 'add':
            fc_input_dim = d_model
        else:
            raise ValueError("fusion_mode must be 'concat' or 'add'")

        self.fc = nn.Sequential(
            nn.Linear(fc_input_dim, d_model // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, 1)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, wind_history, weather_future):
        # wind_history: (batch_size, 288)
        # weather_future: (batch_size, 24, 8)

        x_wind = wind_history.unsqueeze(1)  # (B, 1, 288)
        x_wind = self.tcn_wind(x_wind)     # (B, D, 288)
        x_wind = x_wind[:, :, -1]          

        x_weather = weather_future.transpose(1, 2)  # (B, 8, 24)
        x_weather = self.tcn_weather(x_weather)     # (B, D, 24)
        x_weather = x_weather[:, :, -1]             # -> (B, D)

        if self.fusion_mode == 'concat':
            combined = torch.cat((x_wind, x_weather), dim=1)  # (B, 2D)
        elif self.fusion_mode == 'add':
            combined = x_wind + x_weather  

        output = self.fc(combined)         # (B, 1)
        output = self.sigmoid(output)
        return output
    
tar_model_path4 = 'wind_tcn_caiso_sigmoid_v2.pth'
tar_model4 = WindPowerPredictorTCN_V2()
state_dict = torch.load(tar_model_path4, map_location=device)
tar_model4.load_state_dict(state_dict)
tar_model4.to(device)

In [ ]:
tar_model_list = [tar_model1, tar_model2, tar_model3, tar_model4]

### surrogate model

In [ ]:
class WindPowerPredictor(nn.Module):  
    def __init__(self):  
        super(WindPowerPredictor, self).__init__()  
        self.gru_wind = nn.GRU(input_size=1, hidden_size=50, num_layers=2, batch_first=True)  
        self.gru_weather = nn.GRU(input_size=8, hidden_size=50, num_layers=2, batch_first=True)  
        self.fc = nn.Linear(100, 1)  
        self.sigmoid = nn.Sigmoid()
      
    def forward(self, wind_history, weather_future):  
        wind_history = wind_history.unsqueeze(-1)  
        _, hn_wind = self.gru_wind(wind_history)  
        _, hn_weather = self.gru_weather(weather_future)  
        hn_wind = hn_wind[-1, :, :]  
        hn_weather = hn_weather[-1, :, :]  
        combined = torch.cat((hn_wind, hn_weather), dim=1)  
        output = self.fc(combined)  
        output = self.sigmoid(output)
        return output  
    
sur_model_path1 = 'wind_gru_caiso_sigmoid.pth'
sur_model1 = load_model(sur_model_path1, device)

In [ ]:
class WindPowerPredictor(nn.Module):
    def __init__(self, d_model=50):
        super(WindPowerPredictor, self).__init__()
        self.d_model = d_model
        self.embedding_wind = nn.Linear(1, d_model)
        self.embedding_weather = nn.Linear(8, d_model)
        self.transformer_wind = nn.Transformer(
            d_model=d_model, nhead=2, num_encoder_layers=2, num_decoder_layers=2, dim_feedforward=200, dropout=0.1
        )
        self.transformer_weather = nn.Transformer(
            d_model=d_model, nhead=2, num_encoder_layers=2, num_decoder_layers=2, dim_feedforward=200, dropout=0.1
        )
        self.fc = nn.Linear(d_model * 2, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, wind_history, weather_future):
        wind_history = self.embedding_wind(wind_history.unsqueeze(-1))  # (batch_size, seq_len, d_model)
        wind_history = wind_history.permute(1, 0, 2)  # (seq_len, batch_size, d_model)
        weather_future = self.embedding_weather(weather_future)  # (batch_size, seq_len, d_model)
        weather_future = weather_future.permute(1, 0, 2)  # (seq_len, batch_size, d_model)

        transformer_output_wind = self.transformer_wind(wind_history, wind_history)
        transformer_output_weather = self.transformer_weather(weather_future, weather_future)
        
        combined = torch.cat((transformer_output_wind[-1, :, :], transformer_output_weather[-1, :, :]), dim=1)
        output = self.fc(combined)
        output = self.sigmoid(output)  
        return output
    
sur_model_path2 = 'wind_caiso_transformer_sigmoid.pth'
sur_model2 = load_model(sur_model_path2, device)

In [ ]:
class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=(kernel_size-1) * dilation_size, dilation=dilation_size),
                       nn.ReLU(),
                       nn.Dropout(dropout)]
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

class WindPowerPredictorTCN(nn.Module):
    def __init__(self):
        super(WindPowerPredictorTCN, self).__init__()
        self.tcn_wind = TemporalConvNet(num_inputs=1, num_channels=[50]*3, kernel_size=3, dropout=0.2)
        self.tcn_weather = TemporalConvNet(num_inputs=8, num_channels=[50]*3, kernel_size=3, dropout=0.2)
        self.fc = nn.Linear(50 * 2, 1)  # Combined output size of TCNs
        self.sigmoid = nn.Sigmoid()

    def forward(self, wind_history, weather_future):
        wind_history = wind_history.unsqueeze(1)  # (batch_size, 1, seq_len)
        tcn_output_wind = self.tcn_wind(wind_history).transpose(1, 2)[:, -1, :]
        
        weather_future = weather_future.transpose(1, 2)  # (batch_size, 8, seq_len)
        tcn_output_weather = self.tcn_weather(weather_future).transpose(1, 2)[:, -1, :]
        
        combined = torch.cat((tcn_output_wind, tcn_output_weather), dim=1)
        output = self.fc(combined)
        output = self.sigmoid(output)
        return output
    
sur_model3 = WindPowerPredictorTCN()
sur_model_path3 = 'wind_tcn_caiso_sigmoid.pth'
sur_model3.load_state_dict(torch.load(sur_model_path3))
sur_model3.to(device)

In [ ]:
class WindPowerPredictor(nn.Module):
    def __init__(self):
        super(WindPowerPredictor, self).__init__()
        self.lstm_wind = nn.LSTM(input_size=1, hidden_size=50, num_layers=2, batch_first=True)
        self.lstm_weather = nn.LSTM(input_size=8, hidden_size=50, num_layers=2, batch_first=True)
        self.fc = nn.Linear(100, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, wind_history, weather_future):
        wind_history = wind_history.unsqueeze(-1)
        _, (hn_wind, _) = self.lstm_wind(wind_history)
        _, (hn_weather, _) = self.lstm_weather(weather_future)
        hn_wind = hn_wind[-1, :, :]
        hn_weather = hn_weather[-1, :, :]
        combined = torch.cat((hn_wind, hn_weather), dim=1)
        output = self.fc(combined)
        output = self.sigmoid(output)  
        return output
    
sur_model_path4 = 'wind_caiso_lstm_sigmoid.pth'
sur_model4 = WindPowerPredictor()
sur_model4.load_state_dict(torch.load(sur_model_path4))
sur_model4.to(device)

In [ ]:
sur_model_list = [sur_model1, sur_model2, sur_model3, sur_model4]

## Attack peroformance

### RA

In [ ]:
def adversarial_attack_RA(model, data_loader, epsilon=0.01, device='cpu'):
    model.eval()  
    all_preds = []
    all_adversarial_preds = []
    all_targets = []
    all_original_weather = []
    all_adversarial_weather = []
    
    for parameters in model.parameters():
        parameters.requires_grad = False
    
    for wind_history, weather_future, future_wind_power in data_loader:
        wind_history = wind_history.to(device)
        weather_future = weather_future.to(device)
        future_wind_power = future_wind_power.to(device)
        original_output = model(wind_history, weather_future)
        uap=torch.empty_like(weather_future).uniform_(-epsilon, epsilon)
        adversarial_weather = weather_future +uap
        adversarial_weather = torch.clamp(adversarial_weather, 0, 1)  
        adversarial_output = model(wind_history, adversarial_weather.detach())  
        all_preds.append(original_output.detach().cpu().numpy())
        all_adversarial_preds.append(adversarial_output.detach().cpu().numpy())
        all_targets.append(future_wind_power.cpu().numpy())
        all_original_weather.append(weather_future.detach().cpu().numpy())
        all_adversarial_weather.append(adversarial_weather.detach().cpu().numpy())

    model.eval()  

    return all_preds, all_adversarial_preds, all_targets, all_original_weather, all_adversarial_weather

In [ ]:
epsilon_list=[0.03,0.05,0.1]
for epsilon in epsilon_list:
    for tar_model in tar_model_list:
        original_preds, adversarial_preds, targets, original_weathers, adversarial_weathers = adversarial_attack_RA(tar_model, test_loader, epsilon=epsilon, device=device)
        all_targets = np.concatenate(targets).flatten()
        all_adversarial_preds=np.concatenate(adversarial_preds).flatten()
        all_targets_inv=wind_power_scaler.inverse_transform(np.array(all_targets).reshape(-1, 1)).squeeze()
        all_preds_inv=wind_power_scaler.inverse_transform(np.array(all_adversarial_preds).reshape(-1, 1)).squeeze()
        rmse = mean_squared_error(all_targets_inv, all_preds_inv, squared=False)
        mse = mean_squared_error(all_targets_inv, all_preds_inv)
        print(f'RMSE_inv: {rmse:.10f}')
        print(f'MSE_inv: {mse:.10f}')
        mae = mean_absolute_error(all_targets_inv, all_preds_inv)
        print(f'MAE_inv: {mae:.10f}')
        print('---------------------------------------------------')

### FGSM

In [ ]:
def adversarial_attack_FGSM(sur_model, model, data_loader, epsilon=0.01, device='cpu'):
    sur_model.train()  
    model.eval()
    all_preds = []
    all_adversarial_preds = []
    all_targets = []
    all_original_weather = []
    all_adversarial_weather = []

    for parameters in sur_model.parameters():
        parameters.requires_grad = False
    
    for wind_history, weather_future, future_wind_power in data_loader:
        wind_history = wind_history.to(device)
        weather_future = weather_future.to(device)
        future_wind_power = future_wind_power.to(device)
        weather_future.requires_grad = True
        original_output = model(wind_history, weather_future)
        sur_original_output = sur_model(wind_history, weather_future)
        original_loss = torch.sum(sur_original_output.squeeze()- future_wind_power)
        sur_model.zero_grad()
        original_loss.backward()  

        # adversarial perturbation
        weather_future_grad = weather_future.grad.data
        adversarial_weather = weather_future + epsilon * weather_future_grad.sign()
        adversarial_weather = torch.clamp(adversarial_weather, 0, 1)  # ensure the data is within a reasonable range
        adversarial_output = model(wind_history, adversarial_weather.detach())  

        all_preds.append(original_output.detach().cpu().numpy())
        all_adversarial_preds.append(adversarial_output.detach().cpu().numpy())
        all_targets.append(future_wind_power.cpu().numpy())
        all_original_weather.append(weather_future.detach().cpu().numpy())
        all_adversarial_weather.append(adversarial_weather.detach().cpu().numpy())

    model.eval()  

    return all_preds, all_adversarial_preds, all_targets, all_original_weather, all_adversarial_weather

In [ ]:
epsilon_list=[0.03,0.05,0.1]
for epsilon in epsilon_list:
    for sur_model in sur_model_list:
        for model in tar_model_list:
            original_preds, adversarial_preds, targets, original_weathers, adversarial_weathers = adversarial_attack_FGSM(sur_model, model, test_loader, epsilon=epsilon, device=device)
            all_targets = np.concatenate(targets).flatten()
            all_adversarial_preds=np.concatenate(adversarial_preds).flatten()
            all_targets_inv=wind_power_scaler.inverse_transform(np.array(all_targets).reshape(-1, 1)).squeeze()
            all_preds_inv=wind_power_scaler.inverse_transform(np.array(all_adversarial_preds).reshape(-1, 1)).squeeze()
            rmse = mean_squared_error(all_targets_inv, all_preds_inv, squared=False)
            mse = mean_squared_error(all_targets_inv, all_preds_inv)
            print(f'RMSE_inv: {rmse:.10f}')
            print(f'MSE_inv: {mse:.10f}')
            mae = mean_absolute_error(all_targets_inv, all_preds_inv)
            print(f'MAE_inv: {mae:.10f}')
            print('---------------------------------------------------')

### PGD

In [ ]:
def adversarial_attack_PGD(sur_model, model, data_loader, epsilon=0.01, alpha=0.003, num_iterations=80, device='cpu'):
    sur_model.train()  
    model.eval()
    all_preds = []
    all_adversarial_preds = []
    all_targets = []
    all_original_weather = []
    all_adversarial_weather = []

    for parameters in sur_model.parameters():
        parameters.requires_grad = False

    for wind_history, weather_future, future_wind_power in data_loader:
        wind_history = wind_history.to(device)
        weather_future = weather_future.to(device)
        future_wind_power = future_wind_power.to(device)
        adversarial_weather = weather_future.clone().detach()
        adversarial_weather.requires_grad = True
        original_output = model(wind_history, weather_future)

        sur_original_output = sur_model(wind_history, weather_future)
        for i in range(num_iterations):
            original_output = model(wind_history, adversarial_weather)
            
            sur_original_output = sur_model(wind_history, adversarial_weather)
            original_loss = torch.sum(sur_original_output.squeeze()- future_wind_power)
            sur_model.zero_grad()
            original_loss.backward()  
            with torch.no_grad():
                weather_future_grad = adversarial_weather.grad.data
                adversarial_weather += alpha * weather_future_grad.sign()
                adversarial_weather = torch.max(torch.min(adversarial_weather, weather_future + epsilon),
                                                weather_future - epsilon)
                adversarial_weather = torch.clamp(adversarial_weather, 0,1)
                adversarial_weather = adversarial_weather.detach().requires_grad_(True)
     
        adversarial_output = model(wind_history, adversarial_weather)
        original_output = model(wind_history, weather_future)

        all_preds.append(original_output.detach().cpu().numpy())
        all_adversarial_preds.append(adversarial_output.detach().cpu().numpy())
        all_targets.append(future_wind_power.cpu().numpy())
        all_original_weather.append(weather_future.detach().cpu().numpy())
        all_adversarial_weather.append(adversarial_weather.detach().cpu().numpy())
    sur_model.eval()  
    
    return all_preds, all_adversarial_preds, all_targets, all_original_weather, all_adversarial_weather

In [ ]:
epsilon_list=[0.03,0.05,0.1]
for epsilon in epsilon_list:
    for sur_model in sur_model_list:
        for model in tar_model_list:
            original_preds, adversarial_preds, targets, original_weathers, adversarial_weathers = adversarial_attack_PGD(sur_model, model, test_loader, epsilon=epsilon, device=device)
            all_targets = np.concatenate(targets).flatten()
            all_adversarial_preds=np.concatenate(adversarial_preds).flatten()
            all_targets_inv=wind_power_scaler.inverse_transform(np.array(all_targets).reshape(-1, 1)).squeeze()
            all_preds_inv=wind_power_scaler.inverse_transform(np.array(all_adversarial_preds).reshape(-1, 1)).squeeze()
            rmse = mean_squared_error(all_targets_inv, all_preds_inv, squared=False)
            mse = mean_squared_error(all_targets_inv, all_preds_inv)
            print(f'RMSE_inv: {rmse:.10f}')
            print(f'MSE_inv: {mse:.10f}')
            mae = mean_absolute_error(all_targets_inv, all_preds_inv)
            print(f'MAE_inv: {mae:.10f}')
            print('---------------------------------------------------')

### AoA

In [ ]:
from captum.attr import IntegratedGradients
import torch.optim as optim

def compute_attention(model, wind_history, weather_input, baseline_type='zero'):
    was_training = model.training
    try:
        model.train()
        if not weather_input.requires_grad:
            weather_input.requires_grad_(True)

        with torch.backends.cudnn.flags(enabled=False):
            output = model(wind_history, weather_input)

        grad_outputs = torch.ones_like(output)
        
        grads = torch.autograd.grad(
            outputs=output,
            inputs=weather_input,
            grad_outputs=grad_outputs,
            create_graph=True
        )[0]

        attention = torch.abs(grads * weather_input)
        return attention.contiguous()

    finally:
        if not was_training:
            model.eval()

def generate_universal_adversarial_samples(
    surrogate_models,
    test_loader,
    device,
    lambda_aoa=1.0,
    lambda_reg=10.0,
    epsilon=0.1,  
    num_steps=10,
    learning_rate=0.01,
    target_bias=0.1, 
    save_path="universal_adversarial_samples.pkl"
):
   
    for model in surrogate_models:
        model.train() 

    adversarial_samples = []

    for batch_idx, (wind_history, future_weather, true_power) in enumerate(test_loader):
        wind_history = wind_history.to(device)
        future_weather = future_weather.to(device)
        true_power = true_power.to(device)

        A_ori_list = []
        for model in surrogate_models:
            A_ori = compute_attention(model, wind_history, future_weather, baseline_type='original')
            A_ori_list.append(A_ori.detach())

        future_weather_adv = future_weather.detach().clone().requires_grad_(True)
        optimizer = optim.Adam([future_weather_adv], lr=learning_rate)

        for step in range(num_steps):
            optimizer.zero_grad()
            total_loss = 0.0

            for i, model in enumerate(surrogate_models):
                A_adv = compute_attention(model, wind_history, future_weather_adv, baseline_type='original')
                A_ori_flat = A_ori_list[i].reshape(A_ori_list[i].size(0), -1)  
                A_adv_flat = A_adv.reshape(A_adv.size(0), -1)
                cos_sim = nn.functional.cosine_similarity(A_ori_flat, A_adv_flat, dim=1)
                L_aoa = -cos_sim.mean()

                pred_adv = model(wind_history, future_weather_adv).squeeze()
                pred_ori = model(wind_history, future_weather).squeeze().detach()
                L_reg = -pred_adv.mean()

                loss = lambda_aoa * L_aoa + lambda_reg * L_reg
                total_loss += loss

            total_loss.backward()
            optimizer.step()

            with torch.no_grad():
                future_weather_adv.data = torch.clamp(
                    future_weather_adv,
                    min=future_weather - epsilon,
                    max=future_weather + epsilon
                )

        adversarial_samples.append({
            'index': batch_idx,
            'wind_history': wind_history.cpu().detach().numpy(),
            'future_weather_original': future_weather.cpu().detach().numpy(),
            'future_weather_adversarial': future_weather_adv.cpu().detach().numpy(),
            'true_power': true_power.cpu().detach().numpy()
        })


        if batch_idx % 100 == 0:
            print(f"Batch {batch_idx}, Loss: {total_loss.item():.4f}")
    return adversarial_samples

In [ ]:
def evaluate_attack_with_mae(surrogate_models, adversarial_samples, device, wind_power_scaler=None):
    results = []
    for model_idx, model in enumerate(surrogate_models):
        model.eval()
        total_bias = 0.0
        total_mae_ori = 0.0
        total_mae_adv = 0.0
        count = 0
        
        for sample in adversarial_samples:
            wind_history = torch.tensor(sample['wind_history'], dtype=torch.float32).to(device)
            future_weather_ori = torch.tensor(sample['future_weather_original'], dtype=torch.float32).to(device)
            future_weather_adv = torch.tensor(sample['future_weather_adversarial'], dtype=torch.float32).to(device)
            true_power = torch.tensor(sample['true_power'], dtype=torch.float32).to(device)
            
            with torch.no_grad():
                pred_ori = model(wind_history, future_weather_ori).squeeze()
                pred_adv = model(wind_history, future_weather_adv).squeeze()
                
                bias = (pred_adv - pred_ori).mean().item()
                total_bias += bias
                
                mae_ori = torch.abs(pred_ori - true_power).mean().item()
                mae_adv = torch.abs(pred_adv - true_power).mean().item()
                total_mae_ori += mae_ori
                total_mae_adv += mae_adv
                
                count += 1
        
        avg_bias = total_bias / count
        avg_mae_ori = total_mae_ori / count
        avg_mae_adv = total_mae_adv / count
        mae_increase = avg_mae_adv - avg_mae_ori
        
        result = {
            'model_idx': model_idx,
            'avg_prediction_bias': avg_bias,
            'avg_mae_original': avg_mae_ori,
            'avg_mae_adversarial': avg_mae_adv,
            'mae_increase': mae_increase
        }
        results.append(result)
        
        print(f"Model {model_idx}:")
        print(f"adversarial MAE = {avg_mae_adv:.4f}")
    return results

In [ ]:
def generate_aoa_adversarial_samples(
    surrogate_models,
    test_loader,
    device,
    epsilon=0.1,
    num_steps=80,
    learning_rate=0.003,
    lambda_aoa=1.0,
    lambda_reg=10.0,
):
    for model in surrogate_models:
        model.train()  
        for param in model.parameters():
            param.requires_grad_(False)

    adversarial_samples = []

    for batch_idx, (wind_history, future_weather, true_power) in enumerate(test_loader):
        wind_history = wind_history.to(device)
        future_weather = future_weather.to(device)
        true_power = true_power.to(device)

        # Compute original attention from all surrogate models
        A_ori_list = []
        for model in surrogate_models:
            A_ori = compute_attention(model, wind_history, future_weather, baseline_type='original')
            A_ori_list.append(A_ori.detach())

        # Initialize adversarial input
        future_weather_adv = future_weather.clone().detach().requires_grad_(True)
        optimizer = torch.optim.Adam([future_weather_adv], lr=learning_rate)

        for step in range(num_steps):
            optimizer.zero_grad()
            total_loss = 0.0

            for i, model in enumerate(surrogate_models):
                A_adv = compute_attention(model, wind_history, future_weather_adv, baseline_type='original')
                A_ori_flat = A_ori_list[i].reshape(A_ori_list[i].size(0), -1)
                A_adv_flat = A_adv.reshape(A_adv.size(0), -1)
                cos_sim = torch.nn.functional.cosine_similarity(A_ori_flat, A_adv_flat, dim=1)
                L_aoa = -cos_sim.mean()

                pred_adv = model(wind_history, future_weather_adv).squeeze()
                L_reg = -pred_adv.mean()  # maximize prediction

                loss = lambda_aoa * L_aoa + lambda_reg * L_reg
                total_loss += loss

            total_loss.backward()
            optimizer.step()

            with torch.no_grad():
                future_weather_adv.clamp_(min=future_weather - epsilon, max=future_weather + epsilon)
                future_weather_adv.clamp_(0.0, 1.0)

        adversarial_samples.append({
            'wind_history': wind_history.detach().cpu().numpy(),
            'future_weather_original': future_weather.detach().cpu().numpy(),
            'future_weather_adversarial': future_weather_adv.detach().cpu().numpy(),
            'true_power': true_power.detach().cpu().numpy()
        })

    return adversarial_samples

In [ ]:
def evaluate_aoa_on_target_model(
    target_model,
    adversarial_samples,
    device,
    wind_power_scaler
):
    target_model.eval()
    all_targets = []
    all_preds_original = []
    all_preds_adversarial = []

    for sample in adversarial_samples:
        wind_history = torch.tensor(sample['wind_history'], dtype=torch.float32).to(device)
        weather_orig = torch.tensor(sample['future_weather_original'], dtype=torch.float32).to(device)
        weather_adv = torch.tensor(sample['future_weather_adversarial'], dtype=torch.float32).to(device)
        true_power = torch.tensor(sample['true_power'], dtype=torch.float32).to(device)

        with torch.no_grad():
            pred_orig = target_model(wind_history, weather_orig).squeeze()
            pred_adv = target_model(wind_history, weather_adv).squeeze()

        all_targets.append(true_power.cpu().numpy())
        all_preds_original.append(pred_orig.cpu().numpy())
        all_preds_adversarial.append(pred_adv.cpu().numpy())

    all_targets = np.concatenate(all_targets).flatten()
    all_preds_orig = np.concatenate(all_preds_original).flatten()
    all_preds_adv = np.concatenate(all_preds_adversarial).flatten()

    # Inverse transform to original scale
    all_targets_inv = wind_power_scaler.inverse_transform(all_targets.reshape(-1, 1)).squeeze()
    all_preds_orig_inv = wind_power_scaler.inverse_transform(all_preds_orig.reshape(-1, 1)).squeeze()
    all_preds_adv_inv = wind_power_scaler.inverse_transform(all_preds_adv.reshape(-1, 1)).squeeze()

    mae_orig = mean_absolute_error(all_targets_inv, all_preds_orig_inv)
    mae_adv = mean_absolute_error(all_targets_inv, all_preds_adv_inv)
    rmse_adv = mean_squared_error(all_targets_inv, all_preds_adv_inv, squared=False)

    return {
        'mae_original': mae_orig,
        'mae_adversarial': mae_adv,
        'rmse_adversarial': rmse_adv,
        'mae_increase': mae_adv - mae_orig
    }

In [ ]:
epsilon_list = [0.03, 0.05, 0.10]

surrogate_models_for_aoa = sur_model_list  

for epsilon in epsilon_list:
    print(f"\n{'='*60}")
    print(f" AoA Attack (Multi-Surrogate) | ε = {epsilon}")
    print(f"{'='*60}")
    
    aoa_samples = generate_aoa_adversarial_samples(
        surrogate_models=surrogate_models_for_aoa,  
        test_loader=test_loader,
        device=device,
        epsilon=epsilon,
        num_steps=80,
        learning_rate=0.003,
        lambda_aoa=1.0,
        lambda_reg=10.0
    )

    for tar_model in tar_model_list:
        metrics = evaluate_aoa_on_target_model(
            tar_model, aoa_samples, device, wind_power_scaler
        )
        print(f"MAE_adv: {metrics['mae_adversarial']:.6f}")
        print("-" * 50)

## UP/RUP/RUPW

In [ ]:
def adversarial_attack_inde(uap_loaded, model, data_loader, device='cpu'):
    model.eval()  
    all_preds = []
    all_adversarial_preds = []
    all_targets = []
    all_original_weather = []
    all_adversarial_weather = []
    for wind_history, weather_future, future_wind_power in data_loader:
        wind_history = wind_history.to(device)
        weather_future = weather_future.to(device)
        future_wind_power = future_wind_power.to(device)
        original_output = model(wind_history, weather_future)
        adversarial_weather = weather_future + uap_loaded
        adversarial_weather = torch.clamp(adversarial_weather, 0, 1)  
        adversarial_output = model(wind_history, adversarial_weather.detach())  
        all_preds.append(original_output.detach().cpu().numpy())
        all_adversarial_preds.append(adversarial_output.detach().cpu().numpy())
        all_targets.append(future_wind_power.cpu().numpy())
        all_original_weather.append(weather_future.detach().cpu().numpy())
        all_adversarial_weather.append(adversarial_weather.detach().cpu().numpy())
    model.eval()  
    return all_preds, all_adversarial_preds, all_targets, all_original_weather, all_adversarial_weather


def evaluate_uap_on_target(uap_tensor, target_model, data_loader, scaler, device):
    _, adv_preds, targets, _, _ = adversarial_attack_inde(
        uap_tensor, target_model, data_loader, device=device
    )
    
    adv_flat = np.concatenate(adv_preds).flatten()
    tgt_flat = np.concatenate(targets).flatten()

    tgt_inv = scaler.inverse_transform(tgt_flat.reshape(-1, 1)).squeeze()
    pred_inv = scaler.inverse_transform(adv_flat.reshape(-1, 1)).squeeze()
    
    mae = mean_absolute_error(tgt_inv, pred_inv)
    rmse = mean_squared_error(tgt_inv, pred_inv, squared=False)
    r2 = r2_score(tgt_inv, pred_inv)
    
    return {'mae': mae, 'rmse': rmse, 'r2': r2}


In [ ]:
epsilons = [('003', 0.03), ('005', 0.05), ('010', 0.10)]
model_names = ['tcn', 'lstm', 'gru', 'transformer']

for suffix, epsilon_val in epsilons:
    print(f"\n{'='*60}")
    print(f"Universal Attacks | ε = {epsilon_val}")
    print(f"{'='*60}")

    # === Load all UAPs (flattened across models) ===
    all_uaps = []
    for name in model_names:
        pkl_path = f'uap_results_{name}_epsilon_{suffix}.pkl'
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            uap_list = data['uap_list']
            for uap_np in uap_list:
                if isinstance(uap_np, torch.Tensor):
                    uap_np = uap_np.cpu().numpy()
                uap_tensor = torch.tensor(uap_np, dtype=torch.float32).to(device)
                all_uaps.append(uap_tensor)

    # === Load RUP ===
    with open(f'rup_ensemble_{suffix}_caiso.pkl', 'rb') as f:
        rup_np = pickle.load(f)
    if isinstance(rup_np, torch.Tensor):
        rup_np = rup_np.cpu().numpy()
    rup_tensor = torch.tensor(rup_np, dtype=torch.float32).to(device)

    # === Load RUPW ===
    with open(f'RUPW{suffix}_caiso.pkl', 'rb') as f:
        rupw_np = pickle.load(f)
    if isinstance(rupw_np, torch.Tensor):
        rupw_np = rupw_np.cpu().numpy()
    rupw_tensor = torch.tensor(rupw_np, dtype=torch.float32).to(device)

    # === Evaluate on each target model ===
    for tar_model in tar_model_list:
        # --- Print each UAP individually ---
        for idx, uap_tensor in enumerate(all_uaps):
            metrics = evaluate_uap_on_target(uap_tensor, tar_model, test_loader, wind_power_scaler, device)
            print(f"  UAP #{idx:02d}             | MAE: {metrics['mae']:.6f}, RMSE: {metrics['rmse']:.6f}")

        # --- RUP ---
        rup_metrics = evaluate_uap_on_target(rup_tensor, tar_model, test_loader, wind_power_scaler, device)
        print(f"  RUP (weighted)         | MAE: {rup_metrics['mae']:.6f}, RMSE: {rup_metrics['rmse']:.6f}")

        # --- RUPW ---
        rupw_metrics = evaluate_uap_on_target(rupw_tensor, tar_model, test_loader, wind_power_scaler, device)
        print(f"  RUPW (average)         | MAE: {rupw_metrics['mae']:.6f}, RMSE: {rupw_metrics['rmse']:.6f}")
        
        print("-" * 50)

### execution time

In [ ]:
import time
import torch
import numpy as np

# UP/RUP/RUPW
name='lstm'
suffix='010'
pkl_path = f'uap_results_{name}_epsilon_{suffix}.pkl'
with open(pkl_path, 'rb') as f:
    data = pickle.load(f)
    uap_list = data['uap_list']
uap_loaded = uap_list[0].to(device)

times = []
count = 0

total_start = time.perf_counter()

for wind_history, weather_future, future_wind_power in test_loader:
    count += 1
    iter_start = time.perf_counter()
    wind_history = wind_history.to(device)
    weather_future = weather_future.to(device)
    future_wind_power = future_wind_power.to(device)

    adversarial_weather = weather_future + uap_loaded
    adversarial_weather = torch.clamp(adversarial_weather, 0.0, 1.0)

    iter_end = time.perf_counter()
    times.append(iter_end - iter_start)

total_end = time.perf_counter()

times_ms = np.array(times) * 1000
mean_ms = times_ms.mean()
std_ms = times_ms.std(ddof=1)  

print(f"   UAP : {mean_ms:.2f} ms ± {std_ms:.2f} ms")
print(f"   count: {count}")
print(f"   total time: {total_end - total_start:.3f} seconds")

In [ ]:
# FGSM

epsilon = 0.1  

surrogate_model = sur_model4
surrogate_model.train() 

for p in surrogate_model.parameters():
    p.requires_grad_(False)

times = []
count = 0

total_start = time.perf_counter()

for wind_history, weather_future, future_wind_power in test_loader:
    count += 1
    iter_start = time.perf_counter()
    wind_history = wind_history.to(device)
    weather_future = weather_future.to(device)
    adv_input = weather_future.clone().detach().requires_grad_(True)

    output = surrogate_model(wind_history, adv_input)
    loss = -output.sum()  

    surrogate_model.zero_grad()
    loss.backward()

    with torch.no_grad():
        perturbation = epsilon * adv_input.grad.sign()
        adversarial_weather = weather_future + perturbation
        adversarial_weather = torch.clamp(adversarial_weather, 0.0, 1.0)

    iter_end = time.perf_counter()
    times.append(iter_end - iter_start)

total_end = time.perf_counter()

times_ms = np.array(times) * 1000
mean_ms = times_ms.mean()
std_ms = times_ms.std(ddof=1)

print(f" FGSM: {mean_ms:.3f} ms ± {std_ms:.3f} ms")
print(f" count: {count}")
print(f" total time: {total_end - total_start:.3f} seconds")

In [ ]:
# PGD attack

alpha = 0.005
epsilon = 0.1
num_iterations = 80

surrogate_model = sur_model4
surrogate_model.train() 

for p in surrogate_model.parameters():
    p.requires_grad_(False)

times = []
count = 0

total_start = time.perf_counter()

for wind_history, weather_future, future_wind_power in test_loader:
    count += 1
    iter_start = time.perf_counter()
    wind_history = wind_history.to(device)
    weather_future = weather_future.to(device)
    future_wind_power = future_wind_power.to(device)

    with torch.no_grad():
        original_output = surrogate_model(wind_history, weather_future)
    original_output = original_output.detach()

    adversarial_weather = weather_future.clone().detach().requires_grad_(True)

    for _ in range(num_iterations):
        adversarial_output = surrogate_model(wind_history, adversarial_weather)

        loss = torch.sum(adversarial_output.squeeze() - original_output.squeeze())

        surrogate_model.zero_grad()
        loss.backward()

        with torch.no_grad():
            adversarial_weather += alpha * adversarial_weather.grad.sign()
            adversarial_weather = torch.clamp(
                adversarial_weather,
                min=weather_future - epsilon,
                max=weather_future + epsilon
            )
            adversarial_weather = torch.clamp(adversarial_weather, 0.0, 1.0)
            adversarial_weather = adversarial_weather.detach().requires_grad_(True)

    iter_end = time.perf_counter()
    times.append(iter_end - iter_start)

total_end = time.perf_counter()

times_ms = np.array(times) * 1000
mean_ms = times_ms.mean()
std_ms = times_ms.std(ddof=1)

print(f"   PGD ({num_iterations} iters): {mean_ms:.2f} ms ± {std_ms:.2f} ms")
print(f"   count: {count}")
print(f"   total time: {total_end - total_start:.3f} seconds")

In [ ]:
# RA

times = []
count = 0

total_start = time.perf_counter()

for wind_history, weather_future, future_wind_power in test_loader:
    count += 1
    iter_start = time.perf_counter()
    wind_history = wind_history.to(device)
    weather_future = weather_future.to(device)
    future_wind_power = future_wind_power.to(device)
    noise = torch.randn_like(weather_future)  
    adversarial_weather = weather_future + noise
    adversarial_weather = torch.clamp(adversarial_weather, 0.0, 1.0)

    iter_end = time.perf_counter()
    times.append(iter_end - iter_start)

total_end = time.perf_counter()

times_ms = np.array(times) * 1000
mean_ms = times_ms.mean()
std_ms = times_ms.std(ddof=1)  

print(f"   RA: {mean_ms:.3f} ms ± {std_ms:.3f} ms")
print(f"   count: {count}")
print(f"   total time: {total_end - total_start:.3f} seconds")

In [ ]:
# AOA attack
epsilon = 0.10         
num_steps = 80
learning_rate = 0.003
lambda_aoa = 1.0
lambda_reg = 10.0

surrogate_models = sur_model_list
for model in surrogate_models:
    model.train()
    for p in model.parameters():
        p.requires_grad_(False)

times = []
count = 0
total_start = time.perf_counter()

for wind_history, future_weather, true_power in test_loader:
    count += 1
    iter_start = time.perf_counter()
    
    wind_history = wind_history.to(device)
    future_weather = future_weather.to(device)
    true_power = true_power.to(device)

    # Step 1: Compute original attention (once per batch)
    A_ori_list = []
    for model in surrogate_models:
        A_ori = compute_attention(model, wind_history, future_weather, baseline_type='original')
        A_ori_list.append(A_ori.detach())

    # Step 2: Initialize adversarial input
    future_weather_adv = future_weather.clone().detach().requires_grad_(True)
    optimizer = torch.optim.Adam([future_weather_adv], lr=learning_rate)

    # Step 3: Optimization loop
    for step in range(num_steps):
        optimizer.zero_grad()
        total_loss = 0.0

        for i, model in enumerate(surrogate_models):
            A_adv = compute_attention(model, wind_history, future_weather_adv, baseline_type='original')
            A_ori_flat = A_ori_list[i].reshape(A_ori_list[i].size(0), -1)
            A_adv_flat = A_adv.reshape(A_adv.size(0), -1)
            cos_sim = torch.nn.functional.cosine_similarity(A_ori_flat, A_adv_flat, dim=1)
            L_aoa = -cos_sim.mean()

            pred_adv = model(wind_history, future_weather_adv).squeeze()
            L_reg = -pred_adv.mean()

            loss = lambda_aoa * L_aoa + lambda_reg * L_reg
            total_loss += loss

        total_loss.backward()
        optimizer.step()

        with torch.no_grad():
            future_weather_adv.clamp_(min=future_weather - epsilon, max=future_weather + epsilon)
            future_weather_adv.clamp_(0.0, 1.0)

    iter_end = time.perf_counter()
    times.append(iter_end - iter_start)

total_end = time.perf_counter()

# ====== Output ======
times_ms = np.array(times) * 1000
mean_ms = times_ms.mean()
std_ms = times_ms.std(ddof=1)

print(f"   AoA: {mean_ms:.3f} ms ± {std_ms:.3f} ms")
print(f"   count: {count}")
print(f"   total time: {total_end - total_start:.3f} seconds")